This notebook provides a hands-on introduction to LangChain's core components. We'll build a simple but complete application that demonstrates:
1. Models - Using different LLMs
2. Prompts - Creating and managing prompts
3. Indexes - Working with documents
4. Memory - Maintaining conversation state
5. Chains - Combining components
6. Agents - Adding decision-making

We'll create a "Tour Guide Assistant" that can answer questions about tourist destinations.

In [2]:
# First, install required packages (uncomment if needed)
# !pip install langchain-community langchain-openai python-dotenv tiktoken faiss-cpu wikipedia langchain-huggingface


# langchain-community:
# Contains integrations contributed by the community – such as connectors to various tools (e.g., file loaders, vector stores, APIs).

# langchain-openai:
# Provides OpenAI-specific integrations for LangChain, including support for ChatGPT models, embeddings, and OpenAI tools.

# python-dotenv:
# Loads environment variables from a `.env` file into the Python environment – useful for keeping API keys and secrets secure.

# tiktoken:
# Tokenizer used by OpenAI models – helps you count and manage tokens when working with LLMs to stay within limits.

# faiss-cpu:
# Facebook AI Similarity Search (FAISS) – a library for efficient similarity search and clustering of dense vectors (used in vector databases).

# wikipedia:
# Allows querying and retrieving data from Wikipedia via its API – useful as a knowledge source for RAG (retrieval-augmented generation).

# langchain-huggingface:
# Integrations for using Hugging Face models (e.g., embeddings, LLMs) inside the LangChain ecosystem.


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=f03e940666ac1aeb6db253afc8c1078a0a8f8d6ff9ea7d7054736963fc1a7078
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchai

##1. Setup and Environment

Create a .env file and add your API keys in it


```
HUGGINGFACEHUB_API_TOKEN='your_actual_token_here'
OPENAI_API_KEY='your_actual_token_here'
```



Go to the left sidebar → Files tab (📁 icon) → Click "Upload" → Choose your `.env` file

In [6]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Import LangChain components
from langchain_core.prompts import PromptTemplate  # PromptTemplate allows you to define and format prompts for LLMs
from langchain.memory import ConversationBufferMemory   # Stores previous conversation history in memory to maintain context
from langchain.chains import LLMChain, SequentialChain   # LLMChain runs a single prompt + LLM combination; SequentialChain chains multiple LLMChains together
from langchain_openai import ChatOpenAI  # ChatOpenAI is a wrapper for OpenAI chat models
from langchain.tools import Tool  # Tool is a generic interface for defining external tools (functions, APIs, retrievers) usable by agents
from langchain.tools.retriever import create_retriever_tool  # Creates a retriever tool that can be used by LangChain agents
# AgentType defines types of agents (e.g., `zero-shot-react-description`)
# initialize_agent sets up a LangChain agent using LLM, tools, and memory
from langchain.agents import AgentType, initialize_agent
from langchain_community.tools import WikipediaQueryRun  # WikipediaQueryRun is a tool to query Wikipedia using LangChain interface
from langchain_community.utilities import WikipediaAPIWrapper  # Low-level utility class to access Wikipedia’s API
from langchain_community.document_loaders import WikipediaLoader  # WikipediaLoader loads full articles from Wikipedia as documents (useful for RAG)
from langchain_community.vectorstores import FAISS  # FAISS vector store for storing and searching text embeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter  # RecursiveCharacterTextSplitter breaks documents into chunks
from langchain_huggingface import HuggingFaceEmbeddings   # HuggingFaceEmbeddings provides access to embeddings from Hugging Face models

## 2. Models Component

LangChain supports multiple LLM providers. Let's initialize an OpenAI model.

In [9]:
llm = ChatOpenAI(
        model_name="gpt-4o-mini",      # "gpt-4o-mini" is lightweight and fast, but has token limit 128 (use "gpt-3.5-turbo" on free tier)
        temperature=0.7,               # Controls randomness in responses (0 = deterministic, 1 = creative)
        request_timeout=30             # Max time (in seconds) to wait for a response from the model
    )

# Test the model
print("\nChat LLM:", llm.predict("Tell me a fun fact about Paris in one sentence."))
# The `.predict()` method sends the string to the LLM and gets a response back.


Chat LLM: Paris is home to the world's largest art museum, the Louvre, which houses over 38,000 works of art, including the iconic Mona Lisa.


## 3. Prompts Component

Prompts are templates that structure the input to the LLM.

In [13]:
# Create a prompt template for our tour guide
tour_prompt = PromptTemplate(
    input_variables=["city", "interest"],  # List of variables to be substituted in the template
    template="You are a professional tour guide. Suggest 3 activities in {city} for someone interested in {interest}. Include brief descriptions."
)

# Test the prompt
formatted_prompt = tour_prompt.format(city="Rome", interest="history")  # Use the template to generate a prompt by filling in city and interest
print("Formatted Prompt:\n", formatted_prompt)  # Displays the prompt string that will be sent to the LLM
print("\nLLM Response:\n", llm.predict(formatted_prompt))  # Send the formatted prompt to the LLM and print the response

Formatted Prompt:
 You are a professional tour guide. Suggest 3 activities in Rome for someone interested in history. Include brief descriptions.

LLM Response:
 Certainly! Here are three engaging activities in Rome for history enthusiasts:

1. **Colosseum and Roman Forum Tour**:
   Explore the iconic Colosseum, the largest ancient amphitheater ever built, where gladiators once battled for glory. A guided tour will take you through its fascinating history, including its architectural innovations and the events that took place within its walls. Afterward, stroll through the Roman Forum, the heart of ancient Rome, where you can see the ruins of temples, marketplaces, and civic buildings that once thrived at the center of Roman public life. The juxtaposition of the grandeur and decay here provides a vivid glimpse into the daily lives of Romans.

2. **Vatican Museums and St. Peter’s Basilica**:
   Dive into the rich history of the Catholic Church and Renaissance art with a visit to the Vat

## 4. Indexes Component

Indexes help work with documents and data. We'll use Wikipedia as our data source.

In [14]:
# Let's create a knowledge base about Paris
def create_knowledge_base():
    # Load documents from Wikipedia about "Paris"
    loader = WikipediaLoader(query="Paris", load_max_docs=2)  # `load_max_docs=2` limits how many articles to fetch (e.g., main and related)
    docs = loader.load()  # Loads and returns documents in LangChain document format

    # Split the loaded documents into manageable chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,     # Each chunk will be up to 1000 characters
        chunk_overlap=200    # Overlap between chunks helps preserve context across boundaries
    )
    splits = text_splitter.split_documents(docs)  # Applies the splitter to generate the final list of text chunks

    # Initialize HuggingFace embeddings using an instruction-tuned model for better semantic understanding
    embeddings = HuggingFaceEmbeddings(
        model_name="hkunlp/instructor-xl",   # A powerful embedding model that performs well on semantic search tasks
        model_kwargs={"device": "cuda"}      # Runs the model on GPU (requires GPU runtime; e.g., Google Colab with T4 or A100)
    )

    # Create a FAISS vector store from the document chunks and embeddings
    vectorstore = FAISS.from_documents(splits, embeddings)  # Embeds all text chunks and indexes them for similarity search

    return vectorstore  # Returns the built knowledge base

# Create the knowledge base for Paris
paris_kb = create_knowledge_base()

# Save the FAISS vector store locally for reuse later
paris_kb.save_local("paris_faiss_index")  # Creates a directory with the index and metadata for persistent storage


/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to re

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Later, we can load it like this:
# paris_kb = FAISS.load_local("paris_faiss_index", embeddings)

In [15]:
# Create a retriever
retriever = paris_kb.as_retriever(search_kwargs={"k": 2})

# Converts the FAISS vector store into a retriever
# Sets how many top-matching document chunks the retriever should return for a given query.
# In this case, it will return the top 2 most similar chunks.

In [16]:
# Test the retriever
docs = retriever.get_relevant_documents("Eiffel Tower")
print("Retrieved Documents:")
# Iterate through the retrieved documents and print the first 300 characters of each
for doc in docs:
    print(f"\n{doc.page_content[:300]}...")  # doc.page_content contains the raw text of each matching chunk
                                             # [:300] trims it for readability in output

/tmp/ipython-input-16-257153984.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("Eiffel Tower")


Retrieved Documents:

Paris is a major railway, highway, and air-transport hub served by two international airports: Charles de Gaulle Airport, the third-busiest airport in Europe, and Orly Airport. Paris has one of the most sustainable transportation systems and is one of only two cities in the world that received the S...

Paris (French pronunciation: [paʁi] ) is the capital and largest city of France. With an estimated population of 2,048,472 residents in January 2025 in an area of more than 105 km2 (41 sq mi), Paris is the fourth-most populous city in the European Union and the 30th most densely populated city in th...


## 5. Memory Component

Memory helps maintain conversation state.

In [17]:
# Initialize conversation memory to store previous messages
memory = ConversationBufferMemory(
    memory_key="chat_history",      # Key used to access the conversation history from memory
    return_messages=True            # Ensures the messages are returned as structured objects (not just text strings)
)

# Simulate saving a user message and assistant response into memory
memory.save_context(
    {"input": "Hi! I'm planning a trip to Paris."},           # User input
    {"output": "Great! Paris is wonderful. What would you like to know?"}  # Assistant response
)

# Save another round of conversation to the memory
memory.save_context(
    {"input": "What are some must-see attractions?"},         # User follow-up
    {"output": "The Eiffel Tower, Louvre Museum, and Notre-Dame are top attractions."}  # Assistant reply
)

# Display the entire conversation history stored in memory
print("Current conversation:")
print(memory.load_memory_variables({}))  # Returns a dict with "chat_history" containing the full message list


Current conversation:
{'chat_history': [HumanMessage(content="Hi! I'm planning a trip to Paris.", additional_kwargs={}, response_metadata={}), AIMessage(content='Great! Paris is wonderful. What would you like to know?', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are some must-see attractions?', additional_kwargs={}, response_metadata={}), AIMessage(content='The Eiffel Tower, Louvre Museum, and Notre-Dame are top attractions.', additional_kwargs={}, response_metadata={})]}


/tmp/ipython-input-17-1425393755.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


## 6. Chains Component

Chains combine multiple components together.

In [18]:
# Create a simple LLMChain with a prompt template
chain = LLMChain(
    llm=llm,                      # The LLM to use for generating responses (ChatOpenAI in this case)
    prompt=tour_prompt,          # The custom prompt template that takes 'city' and 'interest' as input
    output_key="activities"      # The key under which the output will be stored (optional, useful when chaining)
)

# Run the chain with specific values for the prompt input variables
result = chain.run(city="Tokyo", interest="technology")  # Replaces {city} with "Tokyo" and {interest} with "technology"

# Print the output from the LLM
print("Tokyo Tech Activities:\n", result)

/tmp/ipython-input-18-3108872419.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=tour_prompt, output_key="activities")
/tmp/ipython-input-18-3108872419.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run(city="Tokyo", interest="technology")


Tokyo Tech Activities:
 Absolutely! Tokyo is a hub of technological innovation and offers a variety of activities for tech enthusiasts. Here are three must-try experiences:

1. **Odaiba's TeamLab Borderless**:
   Step into the mesmerizing world of digital art at TeamLab Borderless, a unique museum located in Odaiba. This interactive art installation features stunning projections that blend art, technology, and nature, allowing visitors to immerse themselves in a captivating environment. The exhibits are ever-changing, as the artworks react to your movements, creating a truly dynamic experience that showcases the fusion of technology and creativity.

2. **Akihabara Electric Town**:
   Known as the mecca for all things electronics and otaku culture, Akihabara is a vibrant district filled with shops specializing in the latest gadgets, anime merchandise, and video games. Explore multi-story electronic stores, where you can find everything from the latest gaming consoles to cutting-edge rob

In [19]:
# Create the first LLMChain: suggests activities based on city and interest
activity_chain = LLMChain(
    llm=llm,                    # Use the ChatOpenAI model
    prompt=tour_prompt,         # Uses the earlier prompt template:
                                # "Suggest 3 activities in {city} for someone interested in {interest}..."
    output_key="activities"     # Stores output under the key 'activities' (used as input for the next chain)
)

# Create a new PromptTemplate for the second step: generating travel tips
tips_prompt = PromptTemplate(
    input_variables=["city", "activities"],  # Inputs needed from previous chain and original input
    template=(
        "Based on these activities in {city}: {activities}. "
        "Provide 3 practical travel tips for visitors."        # New prompt built on first output
    )
)

# Second LLMChain: gives travel tips based on city and the activities from the first chain
tips_chain = LLMChain(
    llm=llm,                    # Reusing the same LLM
    prompt=tips_prompt,        # Uses the new tips_prompt
    output_key="tips"          # Stores output under 'tips'
)

# Combine both chains into a SequentialChain to execute them one after another
overall_chain = SequentialChain(
    chains=[activity_chain, tips_chain],     # Order: first suggest activities, then generate tips
    input_variables=["city", "interest"],    # Inputs needed to start the chain
    output_variables=["activities", "tips"], # Final outputs we want returned
    verbose=True                             # Print detailed execution log while running
)

# Run the complete chain with user input for city and interest
result = overall_chain({"city": "New York", "interest": "art"})

# Print results from both steps
print("\nActivities:", result["activities"])  # Output of first chain
print("\nTips:", result["tips"])              # Output of second chain

/tmp/ipython-input-19-3608441847.py:30: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = overall_chain({"city": "New York", "interest": "art"})




> Entering new SequentialChain chain...

> Finished chain.

Activities: Absolutely! Here are three art-focused activities in New York City that you won't want to miss:

1. **Visit the Metropolitan Museum of Art**: One of the largest and most prestigious art museums in the world, the Met houses over two million works spanning 5,000 years of history. You can explore a vast collection that includes ancient Egyptian artifacts, European masterpieces from the Renaissance, and contemporary art. Don’t miss the stunning rooftop garden for breathtaking views of Central Park and the skyline.

2. **Explore the Chelsea Art Galleries**: This vibrant neighborhood is home to an impressive array of contemporary art galleries. Spend an afternoon wandering through the streets of Chelsea, where you can discover cutting-edge exhibitions from emerging artists and established names alike. Many galleries host opening receptions on Thursday evenings, providing a chance to mingle with artists and other art en

## 7. Agents Component

Agents use LLMs to decide actions to take.

In [21]:
# Create a Wikipedia tool using LangChain's built-in API wrapper
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
# `WikipediaAPIWrapper()` handles the logic of querying Wikipedia and returning summaries

# Define a list of tools the agent can use
tools = [
    Tool(
        name="Wikipedia",                 # Tool name used in prompts or by the agent
        func=wikipedia.run,              # The function to call when this tool is invoked
        description="Useful for looking up general information about cities and landmarks"
        # Helps the agent know when to use this tool based on user queries
    ),
    Tool(
        name="Paris Knowledge Base",     # Custom tool to access our local Paris vector store
        func=lambda query: retriever.get_relevant_documents(query)[0].page_content,
        # A lambda function that runs a similarity search on the Paris retriever and returns the top result’s content
        description="Useful for answering specific questions about Paris (from our knowledge base)"
        # This helps the agent decide to use this instead of the general Wikipedia tool when Paris-specific info is requested
    )
]

Agent Type: CONVERSATIONAL_REACT_DESCRIPTION
* REACT: Reason + Act. The agent can reason about which tool to use before acting.
* Conversational: Maintains memory of previous inputs and responses.
* Description-based: Uses the description fields from each tool to decide which one is best for a query.

In [22]:
# Initialize the agent
agent = initialize_agent(
    tools,                                     # List of tools the agent can use (Wikipedia + Paris KB)
    llm,                                       # The LLM to use (e.g., ChatOpenAI with gpt-4o-mini)
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,  # Agent type that uses reasoning + tool descriptions
    memory=memory,                             # ConversationBufferMemory to track previous messages
    verbose=True                               # Enables step-by-step logging of agent's reasoning and tool usage
)


# Ask the agent a question that should use the Paris knowledge base
print("Agent Response:")
print(agent.run("What's the history of the Eiffel Tower according to your Paris knowledge base?"))
# → Agent will analyze the question, decide to use the "Paris Knowledge Base" tool (based on the description), and respond accordingly

# Ask a follow-up question – memory ensures continuity
print("\nFollow-up Question:")
print(agent.run("What are some similar structures in other cities?"))
# → Agent will remember the context (Eiffel Tower) and use Wikipedia (or both tools) to answer this new question

/tmp/ipython-input-22-3293783206.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


Agent Response:


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Paris Knowledge Base
Action Input: "history of the Eiffel Tower"
Observation: Paris is a major railway, highway, and air-transport hub served by two international airports: Charles de Gaulle Airport, the third-busiest airport in Europe, and Orly Airport. Paris has one of the most sustainable transportation systems and is one of only two cities in the world that received the Sustainable Transport Award twice. Paris is known for its museums and architectural landmarks: the Louvre received 8.9 million visitors in 2023, on track for keeping its position as the most-visited art museum in the world. The Musée d'Orsay, Musée Marmottan Monet and Musée de l'Orangerie are noted for their collections of French Impressionist art. The Pompidou Centre, Musée National d'Art Moderne, Musée Rodin and Musée Picasso are noted for their collections of modern and contemporary art. The historical distri